# Train Detection Model

Git clone and install the [Tensorflow Object Detection API](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) into your working directory (and make sure the directory is untracked).

I used transfer learning with different models from [Tensorflow's detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) to train my detection model. I started with the ssd_inception_v2_coco model for proof of concept with the Tensorflow Object Detection API Tutorial, which notes that it has a nice tradeoff between performance and speed. I also tried the faster_rcnn_resnet101_fgvc model, which is trained on the iNaturalist Species dataset and the faster_rcnn_inception_resnet_v2_atrous_oidv2 model, which is trained on Open Images dataset. The ssd_inception_v2_coco model is trained on the COCO dataset. 

After untracking my pre_trained_model directory from GitHub (the files are big), to get the pipeline for each model I downloaded the * .tar.gz file from Tensorflow's detection model zoo and extracted the contents into a folder in my pre_trained_model directory. 

Each model folder includes a pipeline.config file. For each model, make a copy of the config file, label it by model, and move it to to a new folder named by model, so I used ssd_inception_v2_coco_train/ (also add to .gitignore). The ssd_inception_v2_coco file was out of date so I used [this one](https://github.com/developmentseed/label-maker/blob/94f1863945c47e1b69fe0d6d575caa0b42aa8d63/examples/utils/ssd_inception_v2_coco.config) instead. In the config file, change the number of classes, decrease batch size to whatever your computer can handle (I used 12), and all the places that say PATH_TO_BE_CONFIGURED.

Next copy the train script from the models/research/object_detection/legacy/train.py to the ddb folder and run the following from the ddb folder.

In [ ]:
python train.py --logtostderr --train_dir=<PATH TO MODEL TRAIN DIRECTORY> --pipeline_config_path=<PATH_TO_CONFIG_FILE>


That will initiate the training process and printouts of the step, loss, and seconds per step should appear in the terminal. Depending on computational hardware, it will take different amounts of time to train. The config file is set to run 2,000 steps. It is advised to run model .until TotalLoss is under 2, better to be under 1. A TotalLoss much lower will probably overfit the data. 

Training can be monitored with TensorBoard by opening a new terminal, moving into the ddb folder, and running the following.

In [ ]:
tensorboard --logdir=<MODEL_TRAIN_DIR>\

This starts the TensorBoard server. Then type http://localhost:6006 into your browser to view the model train.

After the model has been trained, export the inference graph by moving the models/research/object_detection/export_inference_graph.py script into the ddb folder. Get the name of the highest numbered model checkpoint file from the model training folder, move to ddb folder, and run the following.

In [ ]:
python export_inference_graph.py --input_type image_tensor --pipeline_config_path <PATH_TO_CONFIG_FILE> --trained_checkpoint_prefix <PATH TO CHECKPOINT> --output_directory trained-inference-graphs/<NAME OUTPUT INFERENCE GRAPH>.pb


Models were evaluated by mean Average Precision using a 50% IOU as the cutoff for true positives. To evaluate the model, edit the config file so that under the eval_config, the number of examples is the size of the test set. Copy the models/research/object_detection/legacy/eval.py to the ddb directory. Then make a directory named by the model_eval (for example ssd_inception_v2_coco_eval), and evaluate by running the following. 

In [ ]:
python eval.py --logtostderr --pipeline_config_path=<PATH_TO_CONFIG_FILE> --checkpoint_dir=<PATH_TO_MODEL_TRAIN> --eval_dir=<PATH_TO_MODEL_EVAL>




To visualize evaluation results in TensorBoard run:

In [ ]:
tensorboard --logdir=eval/

I ran the ssd_inception_v2_coco model locally, and then used a virtual instance on Google Cloud for the others. My results are as follows. 
* ssd_inception_v2_coco = mAP
* faster_rcnn_resnet101_fgvc = mAP
* faster_rcnn_inception_resnet_v2_atrous_oidv2 = mAP